In [2]:

import pandas as pd
import numpy as np
import os
import sys
import difflib

In [3]:
def standardize_district_names():
    """Standard district names for Karnataka"""
    return [
        'Bagalkot', 'Ballari', 'Belagavi', 'Bengaluru Rural', 'Bengaluru Urban',
        'Bidar', 'Chamarajanagar', 'Chikballapur', 'Chikkamagaluru', 'Chitradurga',
        'Dakshina Kannada', 'Davanagere', 'Dharwad', 'Gadag', 'Hassan', 'Haveri',
        'Kalaburagi', 'Kodagu', 'Kolar', 'Koppal', 'Mandya', 'Mysuru', 'Raichur',
        'Ramanagara', 'Shivamogga', 'Tumakuru', 'Udupi', 'Uttara Kannada',
        'Vijayapura', 'Yadgir', 'Vijayanagara'
    ]


In [4]:
def clean_district_name(name):
    """Clean and standardize district names"""
    if pd.isna(name):
        return None

    # Convert to string and clean
    name = str(name).strip()

    # Common name mappings (comprehensive list)
    name_mappings = {
        'Bellary': 'Ballari',
        'Belgaum': 'Belagavi',
        'Bangalore Rural': 'Bengaluru Rural',
        'Bangalore Urban': 'Bengaluru Urban',
        'Bengaluru_Rural': 'Bengaluru Rural',
        'Bengaluru_Urban': 'Bengaluru Urban',
        'Gulbarga': 'Kalaburagi',
        'Bijapur': 'Vijayapura',
        'Shimoga': 'Shivamogga',
        'Tumkur': 'Tumakuru',
        'Mysore': 'Mysuru',
        'DK': 'Dakshina Kannada',
        'UK': 'Uttara Kannada',
        'Dakshina_Kannada': 'Dakshina Kannada',
        'Uttara_Kannada': 'Uttara Kannada',
        'Chikmagalur': 'Chikkamagaluru',
        'Chickmagalur': 'Chikkamagaluru',
        'Chikballapur': 'Chikballapur',
        'Chickballapur': 'Chikballapur'
    }

    # Direct mapping (case-insensitive)
    for old_name, new_name in name_mappings.items():
        if name.lower() == old_name.lower():
            return new_name

    # Check if it's already a standard name
    standard_names = standardize_district_names()
    for standard_name in standard_names:
        if name.lower() == standard_name.lower():
            return standard_name

    # Use difflib for similarity matching
    matches = difflib.get_close_matches(name, standard_names, n=1, cutoff=0.8)
    if matches:
        return matches[0]

    # Handle common patterns
    name_clean = name.replace('_', ' ').replace('-', ' ').title()
    matches = difflib.get_close_matches(name_clean, standard_names, n=1, cutoff=0.8)
    if matches:
        return matches[0]

    return name

In [5]:
def read_and_process_csv(file_path, district_column_name):
    """Read CSV and standardize district names"""
    try:
        # Try different encodings
        encodings = ['utf-8', 'utf-8-sig', 'latin-1', 'cp1252']
        df = None

        for encoding in encodings:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                print(f"Successfully read {file_path} with {encoding} encoding")
                break
            except UnicodeDecodeError:
                continue

        if df is None:
            raise ValueError(f"Could not read {file_path} with any encoding")

        # Clean column names
        df.columns = df.columns.str.strip()

        # Find district column (case-insensitive)
        district_col = None
        for col in df.columns:
            if col.lower().strip() in [district_column_name.lower(), 'district', 'districtname']:
                district_col = col
                break

        if district_col is None:
            print(f"Warning: District column not found in {file_path}. Columns available: {list(df.columns)}")
            return df

        # Standardize district names
        df['District_Standardized'] = df[district_col].apply(clean_district_name)

        # Remove rows with null standardized districts
        initial_rows = len(df)
        df = df.dropna(subset=['District_Standardized'])
        final_rows = len(df)

        if initial_rows != final_rows:
            print(f"Removed {initial_rows - final_rows} rows with invalid district names from {file_path}")

        return df

    except Exception as e:
        print(f"Error reading {file_path}: {str(e)}")
        return None

In [6]:
def combine_csv_files(soil_file, project_file, rainfall_file, output_file='combined_district_data.csv'):
    """Combine three CSV files based on district names"""

    print("Starting CSV combination process...")
    print("=" * 50)

    # Read soil moisture data
    print("Reading soil moisture data...")
    soil_df = read_and_process_csv(soil_file, 'DistrictName')
    if soil_df is None:
        return False

    # Read project area data
    print("Reading project area data...")
    project_df = read_and_process_csv(project_file, 'District')
    if project_df is None:
        return False

    # Read rainfall data
    print("Reading rainfall data...")
    rainfall_df = read_and_process_csv(rainfall_file, 'District')
    if rainfall_df is None:
        return False

    print("\nData Summary:")
    print(f"Soil data: {len(soil_df)} rows")
    print(f"Project data: {len(project_df)} rows")
    print(f"Rainfall data: {len(rainfall_df)} rows")

    # Start with soil data as base
    combined_df = soil_df.copy()

    # Merge with project data
    print("\nMerging with project data...")
    combined_df = pd.merge(
        combined_df,
        project_df,
        left_on='District_Standardized',
        right_on='District_Standardized',
        how='outer',
        suffixes=('', '_project')
    )

    # Merge with rainfall data
    print("Merging with rainfall data...")
    combined_df = pd.merge(
        combined_df,
        rainfall_df,
        left_on='District_Standardized',
        right_on='District_Standardized',
        how='outer',
        suffixes=('', '_rainfall')
    )

    # Clean up duplicate district columns
    district_columns = [col for col in combined_df.columns if 'district' in col.lower() and col != 'District_Standardized']
    combined_df = combined_df.drop(columns=district_columns, errors='ignore')

    # Rename standardized district column
    combined_df = combined_df.rename(columns={'District_Standardized': 'District'})

    # Reorder columns to have District first
    cols = ['District'] + [col for col in combined_df.columns if col != 'District']
    combined_df = combined_df[cols]

    # Sort by district name
    combined_df = combined_df.sort_values('District')

    # Save combined data
    combined_df.to_csv(output_file, index=False)

    print(f"\nCombination completed!")
    print(f"Combined data saved to: {output_file}")
    print(f"Total rows in combined data: {len(combined_df)}")
    print(f"Total columns: {len(combined_df.columns)}")

    # Show summary statistics
    print("\nDistrict Coverage:")
    standard_districts = set(standardize_district_names())
    found_districts = set(combined_df['District'].dropna())

    print(f"Standard districts: {len(standard_districts)}")
    print(f"Found districts: {len(found_districts)}")
    print(f"Missing districts: {standard_districts - found_districts}")

    # Show data preview
    print("\nData Preview (first 5 rows):")
    print(combined_df.head())

    # Show column names
    print(f"\nColumn Names ({len(combined_df.columns)}):")
    for i, col in enumerate(combined_df.columns, 1):
        print(f"{i:2d}. {col}")

    return True

In [9]:
def main():
    """Main function to run the CSV combiner"""

    # File paths - update these with your actual file paths
    soil_file = "/content/soil_district_avg.csv"  # Update with actual filename
    project_file = "/content/irrigation.csv"  # Update with actual filename
    rainfall_file = "/content/ka_2024_rainfall_districts.csv"  # Update with actual filename
    output_file = "combined_district_data.csv"

    print("District Data CSV Combiner")
    print("=" * 40)

    # Check if files exist
    files_to_check = [soil_file, project_file, rainfall_file]
    missing_files = [f for f in files_to_check if not os.path.exists(f)]

    if missing_files:
        print("Error: The following files were not found:")
        for file in missing_files:
            print(f"  - {file}")
        print("\nPlease update the file paths in the script or ensure files are in the same directory.")
        return

    # Combine files
    success = combine_csv_files(soil_file, project_file, rainfall_file, output_file)

    if success:
        print(f"\n✅ Success! Combined data saved to '{output_file}'")
    else:
        print("\n❌ Failed to combine files. Please check the error messages above.")


In [10]:
if __name__ == "__main__":
    main()

District Data CSV Combiner
Starting CSV combination process...
Reading soil moisture data...
Successfully read /content/soil_district_avg.csv with utf-8 encoding
Reading project area data...
Successfully read /content/irrigation.csv with utf-8 encoding
Removed 5 rows with invalid district names from /content/irrigation.csv
Reading rainfall data...
Successfully read /content/ka_2024_rainfall_districts.csv with utf-8 encoding

Data Summary:
Soil data: 30 rows
Project data: 30 rows
Rainfall data: 32 rows

Merging with project data...
Merging with rainfall data...

Combination completed!
Combined data saved to: combined_district_data.csv
Total rows in combined data: 34
Total columns: 38

District Coverage:
Standard districts: 31
Found districts: 34
Missing districts: set()

Data Preview (first 5 rows):
          District  Average Soilmoisture Level (at 15cm)  \
0        BANGALORE                              5.535187   
1         Bagalkot                              6.514346   
2         